# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [5]:
import pandas as pd 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [2]:
spark = SparkSession.builder.appName("example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 04:42:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [3]:
data = spark.read.csv('datosTarea.csv',header=True,inferSchema=True)
data.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [8]:
data.filter((data.Name.like('M%'))&(col('Number of employees')>=4000)&(col('Number of employees')<=7000)).select(['Name', 'Number of employees']).show()

+--------------------+-------------------+
|                Name|Number of employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [9]:
data.filter((~data.Name.like('b%'))&(~data.Name.like('s%'))&(~data.Name.like('m%'))&(col('Networth')<=500000)).select(['Name', 'Country','Networth']).show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|      Pacheco-Spears|              Sweden|   88435|
|         Harrell LLC|          Guadeloupe|  251274|
|         Jenkins Inc|        South Africa|  139725|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|          Pineda-Cox|             Bolivia|  150880|
|Baker, Mccann and...|               Kenya|  188370|
|            Hahn PLC|             Belarus|  427224|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  1

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [10]:
from pyspark.sql.functions import pandas_udf

In [12]:
@pandas_udf('long')
def substract_mean(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    avg = series.mean()
    return series - avg
data.withColumn("avg_substraction", substract_mean(data['Networth'])).show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+----------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|avg_substraction|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+----------------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|         -286416|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|        

/opt/conda/envs/myenv/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:224: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [16]:
from pyspark.sql.functions import sum, col

In [20]:
# Aquí no comprendo bien la pregunta, cuando se hace un group by sobre el campo industry todos los demás sufren alguna agregación. Se pide que se den las empresas con el Networth más alto, pero esa información realmente se pierde con la agregación. Voy a sumar todos los Networth y regresar eso.
data.groupBy("Industry").agg(sum("Networth").alias("Networth")).orderBy(col('Networth').desc()).limit(3).show()

+-------------------+--------+
|           Industry|Networth|
+-------------------+--------+
|           Plastics| 2977807|
|           Printing| 2650288|
|Facilities Services| 2629128|
+-------------------+--------+



>Agrupa por industry y calcula el promedio de empleados que tienen

In [21]:
from pyspark.sql.functions import avg, col

In [22]:
data.groupBy("Industry").agg(avg("Number of employees").alias("Number of employees")).show()

+--------------------+-------------------+
|            Industry|Number of employees|
+--------------------+-------------------+
|Primary / Seconda...|  6457.666666666667|
|     Broadcast Media|             2589.0|
|           Wholesale|             5010.0|
|Investment Manage...|             3133.5|
|    Food / Beverages|             9011.0|
|  Gambling / Casinos|             4873.0|
|Logistics / Procu...|             4155.0|
|            Maritime|              769.0|
|            Wireless|             6146.0|
|Education Management|              339.0|
|       Arts / Crafts|             2800.0|
|           Insurance|             1215.0|
|  Financial Services|             5157.0|
|Business Supplies...|             9097.0|
|Consumer Electronics|             5022.0|
|       Public Safety|             5287.0|
|Information Techn...|             3934.0|
|Civic / Social Or...|             2442.0|
|      Consumer Goods|             9069.0|
|Glass / Ceramics ...|             4952.0|
+----------

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [23]:
data.filter(data.Founded > 2000).show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    4|2bFC1Be8a4ce42f|      Holder-Sellers| https://becker.com/|        Turkmenistan|De-engineered sys...|   2004|          Automotive|                921|  105914|         23|  8200|
|    7|219233e8aFF1BC3|      Hansen-Everett|https://www.kidd....|          

### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [25]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [42]:
featureassembler = VectorAssembler(inputCols = ["Number of employees","Networth","stock_price"], outputCol = "vectorized_independent")

In [43]:
output = featureassembler.transform(data)
output.select("vectorized_independent").show()

+----------------------+
|vectorized_independent|
+----------------------+
|  [3498.0,402269.0,...|
|  [4952.0,569480.0,...|
|  [5287.0,608005.0,...|
|  [921.0,105914.0,2...|
|  [7870.0,905049.0,...|
|  [4914.0,565110.0,...|
|  [7832.0,900679.0,...|
|  [4389.0,504734.0,...|
|  [8167.0,939204.0,...|
|  [9698.0,1115270.0...|
|  [7473.0,859394.0,...|
|  [9011.0,1036264.0...|
|  [2862.0,329130.0,...|
|  [9079.0,1044084.0...|
|  [769.0,88435.0,34.0]|
|  [8508.0,978419.0,...|
|  [6986.0,803389.0,...|
|  [4589.0,527735.0,...|
|  [7961.0,915514.0,...|
|  [5984.0,688160.0,...|
+----------------------+
only showing top 20 rows



In [44]:
finalized_data = output.select("vectorized_independent", "profit")

In [45]:
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])

In [46]:
regressor = LinearRegression(featuresCol = 'vectorized_independent', labelCol = 'profit')
regressor = regressor.fit(train_data)

23/12/04 06:14:44 WARN Instrumentation: [0c4a55be] regParam is zero, which might cause numerical instability and overfitting.


In [47]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+----------------------+------+------------------+
|vectorized_independent|profit|        prediction|
+----------------------+------+------------------+
|  [869.0,99934.0,15.0]|  7787| 11802.57598700452|
|  [921.0,105914.0,2...|  8200|11928.465273920458|
|  [1483.0,170545.0,...| 15246|13725.448087281498|
|  [1678.0,192969.0,...| 16713| 11985.84910053909|
|  [1825.0,209874.0,...|  9901|12189.943745728366|
|  [2215.0,254724.0,...|  6746|12202.762569633467|
|  [2785.0,320275.0,...| 14204|13273.546592444014|
|  [2862.0,329130.0,...| 17043|13399.245700935833|
|  [2992.0,344080.0,...| 13089|13445.613594963394|
|  [3122.0,359030.0,...|  5925|13334.125416492494|
|  [3715.0,427224.0,...|  5730|11733.569254655988|
|  [3987.0,458504.0,...| 16020|11747.285720756314|
|  [4589.0,527735.0,...| 16958|13117.752852904838|
|  [4873.0,560395.0,...| 18276| 13478.66139291542|
|  [4942.0,568330.0,...|  9074| 13683.34939459483|
|  [4952.0,569480.0,...| 12001|13714.844537712954|
|  [5130.0,589950.0,...| 12429|

>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [51]:
pandas_preds = pred_results.predictions.toPandas()
print(pandas_preds)
pandas_preds.to_csv('predictions.csv') 

       vectorized_independent  profit    prediction
0      [869.0, 99934.0, 15.0]    7787  11802.575987
1     [921.0, 105914.0, 23.0]    8200  11928.465274
2    [1483.0, 170545.0, 48.0]   15246  13725.448087
3    [1678.0, 192969.0, 27.0]   16713  11985.849101
4    [1825.0, 209874.0, 40.0]    9901  12189.943746
5    [2215.0, 254724.0, 41.0]    6746  12202.762570
6    [2785.0, 320275.0, 20.0]   14204  13273.546592
7    [2862.0, 329130.0, 28.0]   17043  13399.245701
8    [2992.0, 344080.0, 31.0]   13089  13445.613595
9    [3122.0, 359030.0, 24.0]    5925  13334.125416
10   [3715.0, 427224.0, 12.0]    5730  11733.569255
11   [3987.0, 458504.0, 13.0]   16020  11747.285721
12   [4589.0, 527735.0, 11.0]   16958  13117.752853
13   [4873.0, 560395.0, 34.0]   18276  13478.661393
14   [4942.0, 568330.0, 47.0]    9074  13683.349395
15   [4952.0, 569480.0, 49.0]   12001  13714.844538
16   [5130.0, 589950.0, 12.0]   12429  13129.422999
17   [5287.0, 608005.0, 26.0]   17121  13349.227180
18   [6874.0